# WCA Results Database - Automated Download and Import

Created by Michael George (AKA Logiqx)

Download the latest database extract from https://www.worldcubeassociation.org/export/results

## Initialisation

Basic approach to determine the project directory

In [1]:
import os, sys

projdir = os.path.realpath(os.path.join(sys.path[0], '..'))

## Import Common Libraries

Import the libraries that are used throughout this notebook

In [2]:
import time
import io
import shutil

## Determine the Database Details

Connection details for MySQL / MariaDB database

Note: You will need to specify the password in $HOME/.my.cnf

In [3]:
hostname = os.environ['MYSQL_HOST']
database = os.environ['MYSQL_DATABASE']
username = os.environ['MYSQL_USER']

## Download the ZIP

Save the ZIP to the local machine.

In [4]:
# Hack to force IPv4 - required on my Windows laptop for Alpine 3.13 (and newer)
import Force_IPv4

# The library urllib2 will be used for the download
import urllib.request

# Start time in fractional seconds
pc1 = time.perf_counter()

# Create file handle for the ZIP
zip_url = "https://www.worldcubeassociation.org/export/results/v2/sql"
req = urllib.request.Request(zip_url, headers={'User-Agent': 'Mozilla'})
infile = urllib.request.urlopen(req, timeout = 900)
    
# Write the ZIP to a local file
zip_fn = "WCA_export.zip"
with open(zip_fn, "wb") as outfile:
    shutil.copyfileobj(infile, outfile)

# Close the URL
infile.close()

# End time in fractional seconds
pc2 = time.perf_counter()

print("Download completed in %0.2f seconds" % (pc2 - pc1))

Download completed in 70.10 seconds


## Extract the SQL

Extract the SQL script from within the ZIP file.

In [5]:
# Use the zipfile library to handle the zipfile
import zipfile

# Start time in fractional seconds
pc1 = time.perf_counter()

# Open the ZIP file
with zipfile.ZipFile(zip_fn, mode="r") as zipFile:

    # Identify the SQL script
    for filename in zipFile.namelist():
        if filename.endswith(".sql"):

            # Open the SQL script, so that it can be fixed on the fly, rather than a simple extract
            with zipFile.open(filename, mode="r") as inFile:
                with open(filename, mode="w", encoding="utf-8") as outFile:
                    skip = False

                    for line in io.TextIOWrapper(inFile, encoding="utf-8"):

                        # Avoid backslash issue that was introduced 2024-05-17 */
                        line = line.replace('999999\\- enable the sandbox mode', '999999 - enable the sandbox mode')
                        
                        # Skip records for result_attempts, due to their size
                        if 'INSERT INTO `result_attempts` VALUES' in line:
                            skip = True
                        # Detect end of records for result_attempts
                        elif 'ALTER TABLE' in line or 'UNLOCK TABLES' in line:
                            skip = False

                        if not skip:
                            outFile.write(line)

# End time in fractional seconds
pc2 = time.perf_counter()

print("Extract completed in %0.2f seconds" % (pc2 - pc1))

Extract completed in 20.31 seconds


## Generic SQL Function

Simple function to run a SQL script using the MySQL client

In [6]:
import subprocess

def runSqlScript(source):   
    cmd = ['mysql', '--host=%s' % hostname, '--database=%s' % database, '--user=%s' % username, '--default-character-set=utf8mb4']

    with open(source) as infile:
        proc = subprocess.Popen(cmd, stdin = infile, stdout = subprocess.PIPE, stderr = subprocess.PIPE)
        stdout, stderr = proc.communicate()
        if proc.returncode != 0:
            raise Exception('%s returned %d: %s' % (source, proc.returncode, stderr.decode('utf-8')))

## Populate the WCA Database

Note: The actual database is expected to exist already

In [7]:
# Start time in fractional seconds
pc1 = time.perf_counter()

# Drop legacy tables
runSqlScript(os.path.join(projdir, 'sql', 'drop_tables.sql'))

# Load latest tables
sqlScript = 'WCA_export.sql'
runSqlScript(sqlScript)
os.unlink(sqlScript)

# End time in fractional seconds
pc2 = time.perf_counter()

print("Load completed in %0.2f seconds" % (pc2 - pc1))

Load completed in 72.85 seconds


## Schema Changes

Alter tables and create table indices

In [8]:
# Start time in fractional seconds
pc1 = time.perf_counter()

runSqlScript(os.path.join(projdir, 'sql', 'alter_tables.sql'))
runSqlScript(os.path.join(projdir, 'sql', 'create_indices.sql'))

# End time in fractional seconds
pc2 = time.perf_counter()

print("Indexing completed in %0.2f seconds" % (pc2 - pc1))

Indexing completed in 52.43 seconds


## All Done!